In [1]:
import pandas as pd
import numpy as np

In [2]:
df_events_all = pd.read_csv("all_side_events_complete_lang.csv")
df_events_all

,SE_Code,Code,Title/theme/speakers,Organizer,Date,Title/Description,Speakers,Organizer_Affiliation,Organizer_Name,Organizer_Count,Year,word_count,lang_most,Title/Description_new
0,COP28-000,0,COP 28: Facing the challenges for developing c...,Yvonne Miller Berlie Third World Network (TWN...,"Fri, 01 Dec 2023",COP 28: Facing the challenges for developing c...,Speakers: to be decided,"['Bolivia (Plurinational State of)', 'Third Wo...","['Yvonne Miller Berlie', 'Diego Pacheco']",2.0,2023,20,en,COP 28: Facing the challenges for developing c...
1,COP28-001,1,The Global Stocktake: How to strengthen climat...,Amir Khouzam International Committee of the R...,"Fri, 01 Dec 2023",The Global Stocktake: How to strengthen climat...,Speakers: World Bank Group UNHCR ICRC Somalia ...,"['Somalia', 'International Committee of the Re...","['Amir Khouzam', 'Emily Wilkinson', 'Hafsa Abd...",3.0,2023,59,en,The Global Stocktake: How to strengthen climat...
2,COP28-002,2,Change through participation: Open Dialogues a...,Zuzanna Borowska Polish Ecological Club (PEC)...,"Fri, 01 Dec 2023",Change through participation: Open Dialogues a...,Speakers: A diverse group of speakers from the...,"['REScoop.eu vzw', 'India Water Foundation', '...","['Zuzanna Borowska', 'Olumide Idowu', 'FELI ES...",7.0,2023,57,en,Change through participation: Open Dialogues a...
3,COP28-003,3,Partnerships for Sustainable Finance: Opportun...,Samuel Confidence Dotse HATOF Foundation samu...,"Fri, 01 Dec 2023",Partnerships for Sustainable Finance: Opportun...,Speakers: 1. Minister of Lands and Natural Res...,['Faith Association of the Rehabilitation of S...,"['Samuel Confidence Dotse', 'Samuel Baechie']",2.0,2023,42,en,Partnerships for Sustainable Finance: Opportun...
4,COP28-004,4,Proven climate solutions: following IP&LC lead...,Ashley Emerson Health In Harmony (HIH) ashley...,"Fri, 01 Dec 2023",Proven climate solutions: following IP&LC lead...,"Speakers: Myrna Cunningham, Pawanka Fund Nonet...","['Meridian Institute', 'Rights and Resources I...","['Ashley Emerson', 'Liz Duxbury', 'Lucy Mulenk...",4.0,2023,50,en,Proven climate solutions: following IP&LC lead...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4683,COP09-109,109,Russian Business Forum: Corporations and Regio...,Evgeniy Sokolov National Carbon Sequestration...,"Thu, 11 Dec 2003",Russian Business Forum: Corporations and Regio...,NaN,['National Carbon Sequestration Foundation (NC...,['Evgeniy Sokolov'],1.0,2003,41,en,Russian Business Forum: Corporations and Regio...
4684,COP09-110,110,Standards to ensure high quality LULUCF projec...,Olivia Tanujaya Yayasan Pelangi (PELANGI) oli...,"Thu, 11 Dec 2003",Standards to ensure high quality LULUCF projec...,NaN,['Yayasan Pelangi (PELANGI)'],['Olivia Tanujaya'],1.0,2003,51,en,Standards to ensure high quality LULUCF projec...
4685,COP09-111,111,Linking Article 2 and Article 6: Experiments i...,Jean-Pascal van Ypersele Belgium vanypersele@...,"Fri, 12 Dec 2003",Linking Article 2 and Article 6: Experiments i...,NaN,['Belgium'],['Jean-Pascal van Ypersele'],1.0,2003,49,en,Linking Article 2 and Article 6: Experiments i...
4686,COP09-112,112,South-North Dialogue - Equity in the Greenhous...,"Bernd Brouns Wuppertal Institute for Climate,...","Fri, 12 Dec 2003",South-North Dialogue - Equity in the Greenhous...,NaN,"['Wuppertal Institute for Climate, Environment...",['Bernd Brouns'],1.0,2003,38,en,South-North Dialogue - Equity in the Greenhous...


In [3]:
org_mismatch = pd.read_excel("affiliation/event_affiliation_mismatch.xlsx")
org_mismatch

,original_name,matched_name,Unnamed: 2
0,WMO/UNEP Intergovernmental Panel on Climate Ch...,World Meteorological Organization (WMO),NaN
1,Economic Commission for Africa (ECA),United Nations Economic Commission for Africa ...,NaN
2,United Nations Entity for Gender Equality and ...,UN Women,NaN
3,Southern African Faith Communities Environment...,Southern African Faith Communities' Environmen...,NaN
4,Lund University,Lund University (LU),NaN
...,...,...,...
150,"NGO Centre of Environmental Initiatives ""Ecoac...","NGO Centre of Environmental Initiatives ""Ecoac...",NaN
151,Australian Conservation Foundation (ACF),Australian Conservation Foundation (ACF),NaN
152,"Fundación Desarrollo Sustentable, A.C. (FDS)","Fundación Desarrollo Sustentable, A.C. (FDS)",NaN
153,Union Française de l'Electricité * (UFE),Union Française de l'Electricité (UFE),NaN


In [4]:
# split up detailed titles, desciprions, detailed organizer information
df_events_all[['Title/Description','Speakers']] = df_events_all["Title/theme/speakers"].str.split('  ', expand=True)
df_events_all = df_events_all.set_index("SE_Code").reset_index()
df_events_all["Organizer_Affiliation"] = np.nan
df_events_all["Organizer_Affiliation"] = df_events_all["Organizer_Affiliation"].astype(object)
df_events_all["Organizer_Name"] = np.nan
df_events_all["Organizer_Name"] = df_events_all["Organizer_Name"].astype(object)

# set up edges dataframe
edges_side_events_archive = pd.DataFrame(data={"Title/Description":np.nan,"SE_Code":np.nan},index=[0])

edges_count = 0
for i in df_events_all.index:
    orgnzr = df_events_all["Organizer"][i]
    org_list = []
    organizer_list = []
    for unit in orgnzr.split("  "):
        if "@" in unit:
            org_name = unit.rsplit("@",1)[0]
            org_name = org_name.rsplit(" ",1)[0]
            org_list.append(org_name)
        elif "+" not in unit:
            organizer_list.append(unit)
    df_events_all.at[i,"Organizer_Affiliation"] = list(set(org_list)) # delete duplicates
    df_events_all.at[i,"Organizer_Name"] = organizer_list
    df_events_all.loc[i,"Organizer_Count"] = len(org_list)
    
    org_list_unique = list(set(org_list))
    for n in np.arange(len(org_list_unique))[:-1]:
        source = org_list_unique[n]
        for m in np.arange(len(org_list_unique))[n+1:]:
            edges_side_events_archive.loc[edges_count,"Source_Name"] = source
            edges_side_events_archive.loc[edges_count,"Target_Name"] = org_list_unique[m]
            edges_side_events_archive.loc[edges_count,"Title/Description"] = df_events_all.loc[i,"Title/Description"]
            edges_side_events_archive.loc[edges_count,"SE_Code"] = df_events_all.loc[i,"SE_Code"]
            edges_side_events_archive.loc[edges_count,"Weight"] = 1/(np.arange(len(org_list_unique)).sum())
            edges_count = edges_count + 1
            
edges_side_events_archive["Title/Description"] = edges_side_events_archive["Title/Description"].str.lower()
for i in org_mismatch.index:
    edges_side_events_archive["Source_Name"] = edges_side_events_archive["Source_Name"] .replace(org_mismatch.loc[i,"original_name"],org_mismatch.loc[i,"matched_name"])
    edges_side_events_archive["Target_Name"] = edges_side_events_archive["Target_Name"] .replace(org_mismatch.loc[i,"original_name"],org_mismatch.loc[i,"matched_name"])
#df_events_all.set_index("SE_Code").to_csv("COParchive_side_events_compiled.csv")
edges_side_events_archive#.set_index("SE_Code").to_csv("Edges_Archive.csv")

,Title/Description,SE_Code,Source_Name,Target_Name,Weight
0,cop 28: facing the challenges for developing c...,COP28-000,Bolivia (Plurinational State of),Third World Network (TWN),1.000000
1,the global stocktake: how to strengthen climat...,COP28-001,Somalia,Overseas Development Institute (ODI),0.333333
2,the global stocktake: how to strengthen climat...,COP28-001,Somalia,International Committee of the Red Cross (ICRC),0.333333
3,the global stocktake: how to strengthen climat...,COP28-001,Overseas Development Institute (ODI),International Committee of the Red Cross (ICRC),0.333333
4,change through participation: open dialogues a...,COP28-002,Polish Ecological Club (PEC),Association Femmes Bladi pour le Développement...,0.047619
...,...,...,...,...,...
6901,"the san giorgio group: expanding green, low-em...",COP17-219,"Ca' Foscari University, Venice* (UNIVE)","Climate Policy Initiative, Inc* (CPI)",0.333333
6902,"""the rights of nature, an idea whose time has ...",COP17-220,California College of the Arts* (CCA),Beyond War*,1.000000
6903,post-kyoto offset mechanism and mrv- korean in...,COP17-222,Korean Standards Association* (KSA),Korea Chamber of Commerce and Industry (KCCI),1.000000
6904,low-carbon innovation for an electricity-depen...,COP17-224,"Fundación Desarrollo Sustentable, A.C. (FDS)",Centre for International Governance Innovation...,1.000000


In [5]:
# find unique node names
org_names_source = list(edges_side_events_archive["Source_Name"])
org_names_target = list(edges_side_events_archive["Target_Name"])
org_names = org_names_source + org_names_target
org_names_unique = list(set(org_names))
nodes_side_events = pd.DataFrame(data={"Name":org_names_unique},index=np.arange(len(org_names_unique)))
for i in org_mismatch.index:
    nodes_side_events["Name"] = nodes_side_events["Name"].replace(org_mismatch.loc[i,"original_name"],org_mismatch.loc[i,"matched_name"])
nodes_side_events = nodes_side_events.drop_duplicates().reset_index().drop(columns="index")
nodes_side_events.index.names = ['ID']
nodes_side_events.to_excel("Nodes_All.xlsx")
nodes_side_events = nodes_side_events.reset_index()
nodes_side_events

,ID,Name
0,0,IndustriALL Global Union
1,1,International Forum on Globalization Inc. (IFG)
2,2,London School of Hygiene & Tropical Medicine (...
3,3,United Nations of Youth Network (UNOY)
4,4,BusinessEurope
...,...,...
1777,1777,University of Michigan
1778,1778,United Nations Industrial Development Organiza...
1779,1779,Global Green Growth Institute (GGGI)
1780,1780,National Institute of Advanced Studies (NIAS)


In [6]:
edges_all_withNodes = pd.merge(edges_side_events_archive,nodes_side_events,left_on="Source_Name",right_on="Name")
edges_all_withNodes = edges_all_withNodes.rename(columns={"ID":"Source"}).drop(columns="Name")
edges_all_withNodes = pd.merge(edges_all_withNodes,nodes_side_events,left_on="Target_Name",right_on="Name")
edges_all_withNodes = edges_all_withNodes.rename(columns={"ID":"Target"}).drop(columns="Name")
edges_all_withNodes.set_index("SE_Code").to_excel("Edges_All_withNodes.xlsx")
edges_all_withNodes

,Title/Description,SE_Code,Source_Name,Target_Name,Weight,Source,Target
0,cop 28: facing the challenges for developing c...,COP28-000,Bolivia (Plurinational State of),Third World Network (TWN),1.0,451,1335
1,key priorities of developing countries at cop ...,COP27-002,Bolivia (Plurinational State of),Third World Network (TWN),1.0,451,1335
2,equity is the gateway to climate ambition deve...,COP24-000,South Centre,Third World Network (TWN),1.0,394,1335
3,what to expect at cop 23 the event will featur...,COP23-004,South Centre,Third World Network (TWN),1.0,394,1335
4,expectations for cop 22 the side-event will fe...,COP22-001,South Centre,Third World Network (TWN),1.0,394,1335
...,...,...,...,...,...,...,...
6901,ict solutions for climate mitigation and adapt...,COP17-055,Global E-Sustainability Initiative* (GeSI),TechAmerica Europe,1.0,1544,549
6902,"assessment of biodiversity, forest management,...",COP17-131,Global Biodiversity Information Facility (GBIF),International Union of Forest Research Organiz...,1.0,97,59
6903,leading by example – swedish leadership in glo...,COP17-214,Forum for Reforms and Entrepreneurship* (FORES),University of Gothenburg (GU),1.0,1641,1049
6904,"""the rights of nature, an idea whose time has ...",COP17-220,California College of the Arts* (CCA),Beyond War*,1.0,383,1510
